In [ ]:
import torch
from pathlib import Path
from ibp_neurips19 import models
from ibp_neurips19.attacks import compute_robustness

In [ ]:
# model = 'small'
# dataset = 'CIFAR10'
# epsilon = 0.01
# net = models.fit_to_dataset(models.__dict__[f'{model}_cnn'](), dataset).eval()
# # checkpoint = torch.load(Path.home() / f'results/{dataset}-{model}_cnn-{epsilon}/0.001/checkpoint.pth')
# checkpoint = torch.load(Path.home() / f'results/dm_torch/{dataset.lower()}-{model}-{epsilon}.pth')
# net.load_state_dict(checkpoint['state_dict'])
# # 100 - checkpoint['best_acc1']

In [ ]:
# results = compute_robustness(net, dataset, 'cuda', restarts=1, subset=None, subset_seed=None, attack_kwargs=dict(epsilon=2/255))
# print(results.robustness)
# results.fooling_rate * 100

In [ ]:
import torch
from pathlib import Path
from ibp_neurips19 import models

main_dir = Path('/home/modar/results')
for file in (main_dir / 'pgd').iterdir():
    data = torch.load(file)
    model = data['model']
    dataset = data['dataset']
    epsilon = data['epsilon']
    learning_rate = data['learning_rate']
#     seed = data['seed']
#     subset = data['subset']
#     restarts = data['restarts']
    test_epsilon = data['test_epsilon']
    robustness = data['robustness']
    fooling_rate = data['fooling_rate']
    sorted_errors = data['sorted_errors']
    
    if dataset != 'MNIST' or model != 'small' or epsilon != 0.001 or test_epsilon != 0.1:
        continue

    if learning_rate == 0:
        checkpoint_file = (f'{main_dir}/dm_torch/'
                           f'{dataset.lower()}-{model}-{epsilon}.pth')
    else:
        checkpoint_file = (f'{main_dir}/{dataset}-{model}_cnn-{epsilon}'
                           f'/{learning_rate}/checkpoint.pth')
    checkpoint_file = Path(checkpoint_file)
    checkpoint = torch.load(checkpoint_file)
    accuracy = checkpoint['accuracy'] if learning_rate == 0 else checkpoint['best_acc1'] / 100

    exp = f'{dataset}-{model}-{epsilon}:{test_epsilon:.4e}-{learning_rate}'

#     net = models.__dict__[f'{model}_cnn']()
#     models.fit_to_dataset(net, dataset).eval()
#     net.load_state_dict(checkpoint['state_dict'])
    
    print(f"{exp} [{100 * accuracy:0.2f}%] = {100 * fooling_rate:.2f}%")